<a href="https://colab.research.google.com/github/G-Gaddu/Quant-Material/blob/main/Heston_Option_pricing_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install the additional packages
!pip install nelson-siegel-svensson
!pip install eod

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.1 MB/s eta 0:00:00


In [3]:
# Import the necessary packages
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.optimize import minimize
from datetime import datetime as datetime_CAPI
from eod import EodHistoricalData
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_nss_ols

First we define the characteristic equation

In [4]:
def heston_characteristic(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
  # Define the constants
  a = kappa * theta
  b = kappa + lambd
  rspi = rho*sigma*phi*1j
  # Define the d parameters
  d = np.sqrt((rho*sigma*phi*1j-b)**2 + (phi*1j+phi**2)*sigma**2)
  # Define the g parameter
  g = (b-rspi+d)/(b-rspi-d)
  # Determine the function by it's components
  exp1 = np.exp(r*phi*1j*tau)
  term2 = S0**(phi*1j)*((1-g*np.exp(d*tau))/(1-g))**(-2*a/sigma**2)
  exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2+v0*(b-rspi+d)*((1-np.exp(d*tau))/(1-g*np.exp(d*tau)))/sigma**2)

  return exp1*term2*exp2


Define the integrand as a function

In [5]:
def integrand(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
  args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
  numerator = np.exp(r*tau)*heston_characteristic(phi-1j,*args) - K*heston_characteristic(phi,*args)
  denominator = 1j*phi*K**(1j*phi)
  return numerator/denominator


Perform numerical integration over integrand and calculate the option price

In [6]:
def heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lamd, tau, r):
  args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
  P, umax, N = 0, 100, 10000
  dphi = umax/N

  for i in range(1,N):
    phi = dphi*(2*i+1)/2
    numerator = np.exp(r*tau)*heston_characteristic(phi-1j,*args) - K*heston_characteristic(phi,*args)
    denominator = 1j*phi*K**(1j*phi)
    P += dphi * numerator/denominator

  return np.real((S0-K*np.exp(-r*tau))/2 + P/np.pi)


In [7]:
def heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
  args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
  real_integral, err = np.real(quad(integrand, 0, 100, args=args))
  return (S0 - K*np.exp(-r*tau))/2 + real_integral/np.pi

In [8]:
# Define the parameters to test the model
S0 = 100 # Initial asset price
K = 100 # Strike Price
v0 = 0.1 # Initial volatility
r = 0.03 # Risk free rate
kappa = 1.5768 # Rate of mean reversion of variance process
theta = 0.0398 # Long Term mean variance
sigma = 0.3 # volatility of volatility
lambd = 0.575 # risk premium of variance
rho = -0.5711 # correlation between variance and stock
tau = 1 # time to maturity

In [9]:
heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)

/usr/local/lib/python3.10/dist-packages/scipy/integrate/_quadpack_py.py:611: ComplexWarning: Casting complex values to real discards the imaginary part
  return _quadpack._qagse(func,a,b,args,full_output,epsabs,epsrel,limit)


11.540361819355374

Risk free rate from US Daily Treasury Par Yield Curve Rates

In [10]:
yield_maturities = np.array([1/12, 2/12, 3/12, 6/12, 1, 2, 3, 5, 6, 10, 20, 30])
yields = np.array([0.15, 0.27, 0.50, 0.93, 1.52, 2.13, 2.32, 2.34, 2.37, 2.32, 2.65, 2.52]).astype(float)

In [11]:
curve_fit, status = calibrate_nss_ols(yield_maturities, yields)
curve_fit

NelsonSiegelSvenssonCurve(beta0=2.7274958346334213, beta1=-2.7909066964448366, beta2=-2426.3757519627684, beta3=2429.370130707843, tau1=1.7038350076385094, tau2=1.7020857519390162)

EOD Historical Data API - Get the Market Option prices for the S&P 500 Index

In [12]:
# Load the key from the environment variables
api_key = os.environ.get('EOD_API_KEY') # place the key here
client = EodHistoricalData(api_key)

In [ ]:
resp = client.get_stock_options('GSPC.INDX')

In [ ]:
resp

market_prices = {}

S0 - resp['lastTradePrice']

for i in resp['data']:
  market_prices[i['expirationDate']] = {}
  market_prices[i['expirationDate']]['strikePrice'] = [name['strike'] for name in i['options']['CALL']]
  market_prices[i['expirationDate']]['price'] = [(name['bid']+name['asl'])/2 for name in i['options']['CALL']]

In [ ]:
all_strikes = [v['strike'] for i,v in market_prices.items()]
common_strikes = set.intersection(*map(set, all_strikes))
print('Number of common strikes')
common_strikes = sorted(common_strikes)

In [ ]:
prices = []
maturities = []

for date, v in market_prices.items():
  maturities.append((dt.strptime(date, '%Y-%m-%d')-dt.today()).days.365.25)
  price = [v['price'][i] for i,x in enumerate(v['stirke']) if x in common_strikes]
  prices.append(price)

price_arr = np.array(prices, dtype=object)
np.shape(price_arr)

In [ ]:
volSurface = pd.DataFrame(price_arr, index=maturities, columns=common_strikes)
volSurface = volSurface.iloc[(volSurface.index > 0.04) & (volSurface.index < 1), (volSurface.columns>3000)&(volSurface.columns<5000)]
volSurface

In [ ]:
# Convert the vol surface to dataframe for each option price with parameters
volSurfaceLong = volSurface.melt(ignore_index=False).reset_index()
volSurfaceLong.colun,umns = ['Maturity', 'Strike', 'Price']
# Calculate the risk free rate for each maturity using the fitted yield curve
volSurfaceLong['rate'] = volSurfaceLong['maturity'].apply(curve_fit)

Calibration - Optimization Objective Function

In [ ]:
# Define the variables to be used in optimisation
S0 = resp['lastTradePrice']
K = volSurfaceLong['strike'].to_numpy('float')
r = volSurfaceLong['rate'].to_numpy('float')
P = volSurfaceLong['price'].to_numpy('float')
tau = volSurfaceLong['Maturity'].to_numpy('float')

params = {"v0": {"x0": 0.1, "lbub": [1e-3,0.1]},
          "kappa": {"x0": 3, "lbub": [1e-3,5]},
          "theta": {"x0": 0,05, "lbub": [1e-3,0.1]},
          "sigma": {"x0": 0.3, "lbub":[1e-2,1]},
          "rho": {"x0":-0.8, "lbub":[-1,0]},
          "lambd":{"x0":0.03, "lbub":[-1,1]},
          }

x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]

def SqErr(x):
  v0, kappa, theta, sigma, rho, lambd = [param for param in x]
  err = np.sum((P-heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r))**2/len(P))
  pen = 0
  return err + pen

In [ ]:
result = minimize(SqErr, x0, tol=1e-3, method='SLSQP', options={'maxiter':1e4}, bounds=bnds)

In [ ]:
result

In [ ]:
v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
v0, kappa, theta, sigma, rho, lambd

Calculate Estimated Option Prices Using Calibrated Parameters

In [ ]:
heston_prices = heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)

In [ ]:
volSurfaceLong['heston_price'] = heston_prices

In [ ]:
volSurfaceLong

Visualise Market Prices vs Heston Prices

In [ ]:
import plotly.graph_objects as go
from plotly.graph_objs import Surface
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode

In [ ]:
fig = go.Figure(data=go.Mesh3d(x=volSurfaceLong.maturity),y=volSurfaceLong.strike,z=volSurface.price)
fig.add_scatter3d(x=volSurfaceLong.maturity, y=volSurfaceLong.strike, z=volSurfaceLong.heston_price,mode='markers')
fig.update_layout(
    title.text='Market_Prices vs Calibrated Heston Prices (markers)',
    scene = dict(xaxis_title='TIME (Years)',
                 yaxis_titles='STRIKES (Pts)',
                 zaxis_titles='INDEX OPTION PRICE (Pts)')
    height = 800,
    width = 800
)
fig.show(renderer="colab")